In [ ]:
! git clone https://github.com/MIC-DKFZ/nnUNet.git

In [ ]:
! cd nnUNet; git checkout nnunetv1

In [ ]:
! cd nnUNet; pip install -e .

In [1]:
import os

In [31]:
! nvidia-smi

Thu Dec 14 08:56:15 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050 Ti     Off | 00000000:01:00.0 Off |                  N/A |
| N/A   50C    P8              N/A / ERR! |      8MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [32]:
import torch
import sys
import os
from subprocess import call
print('_____Python, Pytorch, Cuda info____')
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA RUNTIME API VERSION')
#os.system('nvcc --version')
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('_____nvidia-smi GPU details____')
call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('_____Device assignments____')
print('Number CUDA Devices:', torch.cuda.device_count())
print ('Current cuda device: ', torch.cuda.current_device(), ' **May not correspond to nvidia-smi ID above, check visibility parameter')
print("Device name: ", torch.cuda.get_device_name(torch.cuda.current_device()))

_____Python, Pytorch, Cuda info____
__Python VERSION: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
__pyTorch VERSION: 2.1.1+cu121
__CUDA RUNTIME API VERSION
__CUDNN VERSION: 8902
_____nvidia-smi GPU details____
index, name, driver_version, memory.total [MiB], memory.used [MiB], memory.free [MiB]
0, NVIDIA GeForce GTX 1050 Ti, 535.129.03, 4096 MiB, 8 MiB, 4032 MiB
_____Device assignments____
Number CUDA Devices: 1
Current cuda device:  0  **May not correspond to nvidia-smi ID above, check visibility parameter
Device name:  NVIDIA GeForce GTX 1050 Ti


## nnUNet_raw should exists since it is on the github
### Create Manually the other two folders the results and preprocessed

In [33]:
base_dir = os.getcwd()
nnUNet_raw = os.path.join(base_dir,'nnUNet_raw')
nnUNet_preprocessed = os.path.join(base_dir,'nnUNet_preprocessed')
results_folder = os.path.join(base_dir,'nnUNet_results')

In [34]:
print(nnUNet_raw) #Check the directory address is correct

/home/edalita/Documents/MAIA/3-Semestre/MIRMISAProject/IBSR18-tissue-segmentation/nnUNet_raw


In [35]:
os.environ["nnUNet_raw_data_base"] = str(nnUNet_raw)
os.environ["nnUNet_preprocessed"] = str(nnUNet_preprocessed)
os.environ["RESULTS_FOLDER"] = str(results_folder)

## Checking the data and preparing for the trainning

In [36]:
! nnUNet_plan_and_preprocess -t 975 --verify_dataset_integrity



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

Verifying training set
checking case IBSR_01
checking case IBSR_02
checking case IBSR_03
checking case IBSR_04
checking case IBSR_05
checking case IBSR_06
checking case IBSR_07
checking case IBSR_08
checking case IBSR_09
checking case IBSR_10
checking case IBSR_11
checking case IBSR_12
checking case IBSR_13
checking case IBSR_14
checking case IBSR_15
Verifying label values
Expected label values are [0, 1, 2, 3]
Labels OK
Verifying test set
Dataset OK
IBSR_01
IBSR_02
IBSR_03
IBSR_04
IBSR_05
IBSR_06
IBSR_09
IBSR_07
IBSR_10
IBSR_11
IBSR_08
IBSR_12
IBSR_13
IBSR_14
IBSR_15



 Task975_BrainSegmentation
number of threads:  (8, 8) 

usin

## Training
- To train - nnUNet_train TRAINER_CLASS_NAME TASK_NAME_OR_ID FOLD
- To resume - nnUNet_train TRAINER_CLASS_NAME TASK_NAME_OR_ID FOLD -c (just add -c to the training command)
- TRAINER_CLASS_NAME - 2d, 3d_fullres, 3d_lowres, 3d_cascade_fullres
- Everything will be stored in the results folder

In [ ]:
! nnUNet_train -h

In [37]:
! nnUNet_train 2d nnUNetTrainerV2 Task975_BrainSegmentation 3 --npz



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 2d
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  3
modalities:  {0: 'T1'}
use_mask_for_norm OrderedDict([(0, True)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'nonCT')])
stages...

stage:  0
{'batch_size': 69, 'num_pool_per_axis': [5, 5], 'patch_size': array([160, 160]), 'median_patient_size_in_voxels': array([115, 139, 147]), 'current_spacing': array(

## Find the best configuration

In [39]:
! nnUNet_find_best_configuration -m 2d -t 975 -f 3



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

running missing postprocessing for Task975_BrainSegmentation and model 2d
Traceback (most recent call last):
  File "/home/edalita/Documents/MAIA/3-Semestre/MIRMISAProject/IBSR18-tissue-segmentation/projectMisaMira/bin/nnUNet_find_best_configuration", line 33, in <module>
    sys.exit(load_entry_point('nnunet', 'console_scripts', 'nnUNet_find_best_configuration')())
  File "/home/edalita/Documents/MAIA/3-Semestre/MIRMISAProject/IBSR18-tissue-segmentation/nnUNet/nnunet/evaluation/model_selection/figure_out_what_to_submit.py", line 144, in main
    consolidate_folds(output_folder, folds=folds)
  File "/home/edalita/Documents/MAIA/3-

: 

## Predictions

In [ ]:
! nnUNetv2_predict -d Dataset975_BrainSegmentation -i nnUNet_raw/Dataset975_BrainSegmentation/imagesTr -o outTrainning/ -f  3 -tr nnUNetTrainer_5epochs -c 2d -p nnUNetPlans


In [ ]:
! nnUNetv2_apply_postprocessing -i outTrainning/ -o outTrainning/outTrainningpp/ -pp_pkl_file /home/edalita/Documents/MAIA/3-Semestre/MIRMISAProject/IBSR18-tissue-segmentation/nnUNet_results/Dataset975_BrainSegmentation/nnUNetTrainer_5epochs__nnUNetPlans__2d/crossval_results_folds_3/postprocessing.pkl -np 8 -plans_json /home/edalita/Documents/MAIA/3-Semestre/MIRMISAProject/IBSR18-tissue-segmentation/nnUNet_results/Dataset975_BrainSegmentation/nnUNetTrainer_5epochs__nnUNetPlans__2d/crossval_results_folds_3/plans.json

In [ ]:
print(base_dir)

In [ ]:
! nnUNetv2_predict -h


In [ ]:
! nnUNetv2_predict -d Dataset975_BrainSegmentation -i nnUNet_raw/Dataset975_BrainSegmentation/imagesTs -o outTest/ -f  3 -tr nnUNetTrainer_5epochs -c 2d -p nnUNetPlans